In [ ]:
"""
Created on Fri Aug 12 11:43 2022

This script is to produce Figures 4, 7, D1, D2, D3 absed on output of "evalmetrics_results_CV.ipynb"

@author: Clara Burgard
"""

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import matplotlib as mpl
import glob

In [ ]:
%matplotlib qt5

READ IN DATA

In [ ]:
home_path='/bettik/burgardc/'
plot_path = '/bettik/burgardc/PLOTS/generic_plots/'

In [ ]:
inputpath_mask = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/ANTARCTICA_IS_MASKS/nemo_5km_OPM006/'
file_isf_orig = xr.open_dataset(inputpath_mask+'nemo_5km_isf_masks_and_info_and_distance_new_oneFRIS.nc')
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
large_isf = file_isf_nonnan['Nisf'].where(file_isf_nonnan['isf_area_here'] >= 2500, drop=True)
file_isf = file_isf_nonnan.sel(Nisf=large_isf)
file_isf['isf_name'] = file_isf['isf_name'].where(file_isf['isf_name'] != 'Ekstrom', np.array('Ekström', dtype=object))
isf_names = file_isf['isf_name']

In [ ]:
region_list = []
for kisf in file_isf.Nisf:
    if file_isf['isf_name'].sel(Nisf=kisf).values in ['Ross','Nickerson','Sulzberger', 'Cook']:
        region_list.append('East and West Ross')
    elif file_isf['isf_name'].sel(Nisf=kisf).values in ['Filchner','Ronne','Filchner-Ronne']:
        region_list.append('Weddell')        
    elif file_isf['isf_name'].sel(Nisf=kisf).values in ['Ekström','Nivl','Prince Harald','Riiser-Larsen','Fimbul','Roi Baudouin','Lazarev','Stancomb Brunt','Jelbart','Borchgrevink']:
        region_list.append('Dronning Maud Land')
    elif file_isf['isf_name'].sel(Nisf=kisf).values in ['Getz','Thwaites','Crosson','Dotson','Cosgrove','Pine Island']:
        region_list.append('Amundsen')
    elif file_isf['isf_name'].sel(Nisf=kisf).values in ['Venable','George VI','Abbot','Stange','Larsen C','Bach','Larsen D','Wilkins']:
        region_list.append('Peninsula and Bellinghausen')
    elif file_isf['isf_name'].sel(Nisf=kisf).values in ['Amery','Moscow Univ.','Tracy Tremenchus','Totten','West','Shackleton']:
        region_list.append('East Antarctica')
    else:
        print('Argh, help me, '+file_isf['isf_name'].sel(Nisf=kisf).values+' has no region assigned!')
# 6 regions
# regions = ['East and West Ross','Weddell','Dronning Maud Land','Amundsen','Peninsula and Bellinghausen','East Antarctica']
file_isf['region'] = xr.DataArray(data=region_list,dims='Nisf')

In [ ]:
regions = ['Weddell','Peninsula and Bellinghausen','Amundsen','East and West Ross','East Antarctica','Dronning Maud Land']

In [ ]:
# sort ice shelves by regions
nisf_by_reg_list = []
for rr, reg in enumerate(regions):
    subset_isf = file_isf.Nisf.where(file_isf['region']==reg,drop=True)
    nisf_by_reg_list.append(subset_isf.values)
nisf_by_reg_list = np.concatenate(nisf_by_reg_list)

In [ ]:
run_list = ['OPM006','OPM016','OPM018','OPM021']

diff_Gt_CVtime_list = []
diff_box1_CVtime_list = []
diff_Gt_CVisf_list = []
diff_box1_CVisf_list = []

ref_Gt_list = []
ref_box1_list = []
Gt_CVtime_list = []
Gt_CVisf_list = []


for n,nemo_run in enumerate(run_list):

    outputpath_melt = home_path+'DATA/BASAL_MELT_PARAM/processed/MELT_RATE/nemo_5km_'+nemo_run+'/'

    ### READ IN THE REFERENCE
    NEMO_melt_rates_1D = xr.open_dataset(outputpath_melt+'melt_rates_1D_NEMO_oneFRIS.nc')
    ref_Gt = NEMO_melt_rates_1D['melt_Gt_per_y_tot']
    ref_Gt_list.append(ref_Gt)
    NEMO_box1_myr = xr.open_dataset(outputpath_melt+'melt_rates_box1_NEMO_oneFRIS.nc')
    ref_box1 = NEMO_box1_myr['mean_melt_box1_myr']
    ref_box1_list.append(ref_box1)
    
    ### READ IN THE PARAM FILES - CV TIME
    
    # Param files
    outputpath_melt = home_path+'DATA/BASAL_MELT_PARAM/processed/MELT_RATE/nemo_5km_'+nemo_run+'/'
    melt_param_files_CVtime = list(sorted(glob.glob(outputpath_melt+'eval_metrics_1D_*_CVtime.nc')))
    
    # create the list of parameterisation names to fill the param dimension
    param_list = []
    for mfilename in melt_param_files_CVtime:
        paramname = mfilename[91:157].split('.')[0].split('_')[:-2]
        mparam = '_'.join(paramname)
        param_list.append(mparam)
    
    ds_melt_param_CVtime  = xr.open_mfdataset(melt_param_files_CVtime, concat_dim='new_param', combine='nested', coords='minimal',compat='override')#, chunks={'x': chunksize, 'y': chunksize})
    if 'option' in ds_melt_param_CVtime.coords:        
        ds_melt_param_CVtime = ds_melt_param_CVtime.drop('param').drop('option')
    else:
        ds_melt_param_CVtime = ds_melt_param_CVtime.drop('param')
    ds_melt_param_CVtime = ds_melt_param_CVtime.rename({'new_param': 'param'})
    ds_melt_param_CVtime = ds_melt_param_CVtime.assign_coords(param=param_list)
    Gt_CVtime_list.append(ds_melt_param_CVtime['melt_1D_Gt_per_y'])
    diff_Gt_CVtime = ds_melt_param_CVtime['melt_1D_Gt_per_y'] - ref_Gt
    # create new time dimension to concatenate all runs over one time axis
    diff_Gt_CVtime = diff_Gt_CVtime.assign_coords({'time': np.arange(1,len(diff_Gt_CVtime.time)+1)+n*50})
    diff_Gt_CVtime_list.append(diff_Gt_CVtime)
    diff_box1_CVtime = ds_melt_param_CVtime['melt_1D_mean_myr_box1'].mean('time') - ref_box1.mean('time')
    diff_box1_CVtime_list.append(diff_box1_CVtime)
    
    ### READ IN THE PARAM FILES - CV ISF

    melt_param_files_CVisf = list(sorted(glob.glob(outputpath_melt+'eval_metrics_1D_*_CVshelves.nc')))
    ds_melt_param_CVisf  = xr.open_mfdataset(melt_param_files_CVisf, concat_dim='new_param', combine='nested', coords='minimal',compat='override')#, chunks={'x': chunksize, 'y': chunksize})
    if 'option' in ds_melt_param_CVisf.coords:        
        ds_melt_param_CVisf  = ds_melt_param_CVisf.drop('param').drop('option')
    else:
        ds_melt_param_CVisf  = ds_melt_param_CVisf.drop('param')
    ds_melt_param_CVisf = ds_melt_param_CVisf.rename({'new_param': 'param'})
    ds_melt_param_CVisf = ds_melt_param_CVisf.assign_coords(param=param_list)
    Gt_CVisf_list.append(ds_melt_param_CVisf['melt_1D_Gt_per_y'])
    diff_Gt_CVisf = ds_melt_param_CVisf['melt_1D_Gt_per_y'] - ref_Gt
    # create new time dimension to concatenate all runs over one time axis
    diff_Gt_CVisf = diff_Gt_CVisf.assign_coords({'time': np.arange(1,len(diff_Gt_CVisf.time)+1)+n*50})
    diff_Gt_CVisf_list.append(diff_Gt_CVisf)
    diff_box1_CVisf = ds_melt_param_CVisf['melt_1D_mean_myr_box1'].mean('time') - ref_box1.mean('time')
    diff_box1_CVisf_list.append(diff_box1_CVisf)
        
# Merge all either over time or nemo_run
# melt in Gt/yr
Gt_all_CVtime = xr.concat(Gt_CVtime_list, dim='nemo_run')
Gt_all_CVisf = xr.concat(Gt_CVisf_list, dim='nemo_run')
# difference melt Gt/yr and melt nnear grounding line between param and reference
diff_Gt_all_CVtime = xr.concat(diff_Gt_CVtime_list, dim='time')
diff_box1_all_CVtime = xr.concat(diff_box1_CVtime_list, dim='nemo_run')   
diff_Gt_all_CVisf = xr.concat(diff_Gt_CVisf_list, dim='time')
diff_box1_all_CVisf = xr.concat(diff_box1_CVisf_list, dim='nemo_run') 
# reference metrics
ref_Gt_all = xr.concat(ref_Gt_list, dim='nemo_run')
ref_box1_all = xr.concat(ref_box1_list, dim='nemo_run')

In [ ]:
# mean melt near grounding line over all ice shelves
mean_box1_all = ref_box1_all.mean()
mean_box1_all

In [ ]:
# mean integrated melt
tot_Gt = ref_Gt_all.mean('time').mean('nemo_run').mean('Nisf')
tot_Gt

PLOT FIGURE D1

In [ ]:
mean_over_time_Gt = ref_Gt_all.mean('time')
std_over_time_Gt = ref_Gt_all.std('time')
mean_over_time_box1 = ref_box1_all.mean('time')
std_over_time_box1 = ref_box1_all.std('time')

In [ ]:
fig, axs = plt.subplots(1, 2,figsize=(8.24*1.4,8.24/1.1),sharey=True)
isf_names_str = isf_names.values.astype(str).tolist()

for kisf in nisf_by_reg_list[::-1]:

    isf_name_str = isf_names.sel(Nisf=kisf).values.astype(str).tolist()

    for nrun in mean_over_time_Gt.nemo_run:
        
        if nrun == 0:
            ccolor = 'magenta'
        elif nrun == 1:
            ccolor = 'brown'
        elif nrun == 2:
            ccolor = 'orange'
        elif nrun == 3:
            ccolor = 'red'
            
        axs[0].errorbar(mean_over_time_Gt.sel(Nisf=kisf, nemo_run=nrun).values, np.array([isf_name_str]), xerr=std_over_time_Gt.sel(Nisf=kisf, nemo_run=nrun).values,c=ccolor, fmt ='o', markersize=3)
        axs[0].axvline(x=tot_Gt,c='darkgray',linestyle='--',linewidth=0.7)
        axs[1].errorbar(mean_over_time_box1.sel(Nisf=kisf, nemo_run=nrun).values, np.array([isf_name_str]), xerr=std_over_time_box1.sel(Nisf=kisf, nemo_run=nrun).values,c=ccolor, fmt ='o', markersize=3)
        axs[1].axvline(x=mean_box1_all,c='darkgray',linestyle='--',linewidth=0.7)
        
sns.despine()
plt.savefig(plot_path+'reference_melt_with_errorbars.pdf')

PLOT FIGURE 4

In [ ]:
RMSE_Gt_all_CVtime = np.sqrt((diff_Gt_all_CVtime**2).mean(['time','Nisf']))
RMSE_box1_all_CVtime = np.sqrt((diff_box1_all_CVtime**2).mean(['nemo_run','Nisf']))
RMSE_Gt_all_CVisf = np.sqrt((diff_Gt_all_CVisf**2).mean(['time','Nisf']))
RMSE_box1_all_CVisf = np.sqrt((diff_box1_all_CVisf**2).mean(['nemo_run','Nisf']))

In [ ]:
param_list_of_int = ['linear_local', 'quadratic_local', 'quadratic_local_cavslope',
                    'quadratic_local_locslope', 'quadratic_mixed_mean','quadratic_mixed_cavslope',
                    'quadratic_mixed_locslope', 
                    'lazero19_2','lazero19_modif2',
                     'boxes_1_pismyes_picopno','boxes_2_pismyes_picopno','boxes_3_pismyes_picopno','boxes_4_pismyes_picopno',
                     'boxes_3_pismyes_picopyes', 'boxes_4_pismno_picopyes']

param_simple_list = ['linear_local', 'quadratic_local', 'quadratic_local_cavslope',
                   'quadratic_local_locslope', 'quadratic_mixed_cavslope',
                   'quadratic_mixed_locslope', 'quadratic_mixed_mean']
param_plume_list = ['lazero19_2','lazero19_modif2']
param_box_list_hetero =  ['boxes_1_pismyes_picopno','boxes_2_pismyes_picopno','boxes_3_pismyes_picopno','boxes_4_pismyes_picopno'] #'boxes_1_pismno_picopno','boxes_2_pismno_picopno','boxes_3_pismno_picopno','boxes_4_pismno_picopno',
param_picop_list = ['boxes_3_pismyes_picopyes', 'boxes_4_pismno_picopyes']


In [ ]:
# for Tables
for mparam in param_list_of_int:
    print(mparam, RMSE_Gt_all_CVtime.sel(param=mparam).values)


PLOT FIGURE 4

In [ ]:
fig, axs = plt.subplots(1, 2,figsize=(8.24*1.25/1.5,8.24*1.25/2),sharey=True)
for mparam in param_list_of_int[::-1]:
    
    if mparam in param_plume_list:
        ccolor = 'darkorange'
    elif mparam in param_box_list_hetero:
        ccolor = 'purple'
    elif mparam in param_picop_list:
        ccolor = 'maroon'
    elif mparam in param_simple_list:
        ccolor = 'gold'
        
    axs[0].scatter(RMSE_Gt_all_CVtime.sel(param=mparam, profile_domain=50),mparam,marker='x',c='k', s=50)
    axs[1].scatter(RMSE_Gt_all_CVtime.sel(param=mparam, profile_domain=1000),mparam,marker='x',c='k', s=50)
    axs[0].scatter(RMSE_Gt_all_CVtime.sel(param=mparam, profile_domain=50),mparam,marker='o',c=ccolor, s=20)
    axs[1].scatter(RMSE_Gt_all_CVtime.sel(param=mparam, profile_domain=1000),mparam,marker='o',c=ccolor, s=20)

    axs[0].scatter(RMSE_Gt_all_CVisf.sel(param=mparam, profile_domain=50),mparam,marker='+',c='k', s=120)
    axs[1].scatter(RMSE_Gt_all_CVisf.sel(param=mparam, profile_domain=1000),mparam,marker='+',c='k', s=120)
    axs[0].scatter(RMSE_Gt_all_CVisf.sel(param=mparam, profile_domain=50),mparam,marker='o',c=ccolor, s=20)
    axs[1].scatter(RMSE_Gt_all_CVisf.sel(param=mparam, profile_domain=1000),mparam,marker='o',c=ccolor, s=20)

axs[0].set_xlim(20,90)
axs[1].set_xlim(20,90)

sns.despine()
plt.savefig(plot_path+'RMSE_scatter_offshore_CV.pdf')

PLOT FIGURE 7

In [ ]:
fig, axs = plt.subplots(1, 2,figsize=(8.24*1.25/1.5,8.24*1.25/2),sharey=True)
for mparam in param_list_of_int[::-1]:
    
    if mparam in param_plume_list:
        ccolor = 'darkorange'
    elif mparam in param_box_list_hetero:
        ccolor = 'purple'
    elif mparam in param_picop_list:
        ccolor = 'maroon'
    elif mparam in param_simple_list:
        ccolor = 'gold'
        
    if 'profile_domain' in RMSE_Gt_all_CVtime.coords:
        axs[0].scatter(RMSE_Gt_all_CVtime.sel(param=mparam, profile_domain=50),mparam,marker='x',c='k', s=50)
        axs[1].scatter(RMSE_box1_all_CVtime.sel(param=mparam, profile_domain=50),mparam,marker='x',c='k', s=50)
        axs[0].scatter(RMSE_Gt_all_CVtime.sel(param=mparam, profile_domain=50),mparam,marker='o',c=ccolor, s=20)
        axs[1].scatter(RMSE_box1_all_CVtime.sel(param=mparam, profile_domain=50),mparam,marker='o',c=ccolor, s=20)

        axs[0].scatter(RMSE_Gt_all_CVisf.sel(param=mparam, profile_domain=50),mparam,marker='+',c='k', s=120)
        axs[1].scatter(RMSE_box1_all_CVisf.sel(param=mparam, profile_domain=50),mparam,marker='+',c='k', s=120)
        axs[0].scatter(RMSE_Gt_all_CVisf.sel(param=mparam, profile_domain=50),mparam,marker='o',c=ccolor, s=20)
        axs[1].scatter(RMSE_box1_all_CVisf.sel(param=mparam, profile_domain=50),mparam,marker='o',c=ccolor, s=20)

axs[0].set_xlim(20,90)
axs[1].set_xlim(0,2.05)

sns.despine()
plt.savefig(plot_path+'RMSE_scatter_box1_CV.pdf')

HEATMAPS

PLOT FIGURE D2

In [ ]:
plotted_var = diff_Gt_all_CVisf.mean('time').sel(profile_domain=50,param=param_list_of_int,Nisf=nisf_by_reg_list)
plotted_var2 = diff_Gt_all_CVisf.mean('time').sel(profile_domain=1000,param=param_list_of_int,Nisf=nisf_by_reg_list)


x = np.arange(len(plotted_var.param))
y = np.arange(len(plotted_var.Nisf))

fig, axs = plt.subplots(2, 1,figsize=((len(y)+1)/(2*1.5),(len(x)+2)/1.5))
ax0 = axs[0].imshow(plotted_var.values, cmap=plt.cm.coolwarm, vmin=-100, vmax=100)
ax1 = axs[1].imshow(plotted_var2.values, cmap=plt.cm.coolwarm, vmin=-100, vmax=100)
axs[0].set_yticklabels(labels=np.round(RMSE_Gt_all_CVisf.sel(profile_domain=50,param=param_list_of_int).values,1))
axs[1].set_yticklabels(labels=np.round(RMSE_Gt_all_CVisf.sel(profile_domain=1000,param=param_list_of_int).values,1))

for j, dom in enumerate([50, 1000]):
    axs[j].yaxis.tick_right()
    axs[j].set_yticks(x)
    axs[j].yaxis.tick_right()
    if j==0:
        axs[j].xaxis.tick_top()    
        axs[j].set_xticks(y)
        axs[j].set_xticklabels(labels=isf_names.sel(Nisf=plotted_var.Nisf).values, rotation=90)
    else:
        axs[j].set_xticks([])
        axs[j].set_xticklabels(labels=[])

cb_ax = fig.add_axes([0.15, 0.08, 0.7, 0.02])
cbar = fig.colorbar(ax0, cax=cb_ax, extend='both',orientation='horizontal')
fig.savefig(plot_path+'heatmap_meandiff_CVisf_allruns.pdf')

PLOT FIGURE D3

In [ ]:
plotted_var = diff_box1_all_CVisf.mean('nemo_run').sel(profile_domain=50,param=param_list_of_int,Nisf=nisf_by_reg_list)


x = np.arange(len(plotted_var.param))
y = np.arange(len(plotted_var.Nisf))

fig, axs = plt.subplots(1, 1,figsize=((len(y)+1)/4,(len(x)+2)/2.75))
ax0 = axs.imshow(plotted_var.values, cmap=plt.cm.coolwarm, vmin=-3, vmax=3)
axs.set_yticklabels(labels=np.round(RMSE_box1_all_CVisf.sel(profile_domain=50,param=param_list_of_int).values,2))

for j, dom in enumerate([50]):
    axs.yaxis.tick_right()
    axs.set_yticks(x)
    axs.yaxis.tick_right()

    axs.xaxis.tick_top()    
    axs.set_xticks(y)
    axs.set_xticklabels(labels=isf_names.sel(Nisf=plotted_var.Nisf).values, rotation=90)


cb_ax = fig.add_axes([0.15, 0.08, 0.7, 0.02])
cbar = fig.colorbar(ax0, cax=cb_ax, extend='both',orientation='horizontal')
plt.tight_layout()
fig.savefig(plot_path+'heatmap_meandiff_box1_CVisf_allruns.pdf')